# Credmark Modeling Framework Example for Jupyter notebook

## Account - Tokens

version: 2023.4.15


## Initialize

<div class="alert alert-block alert-info">
    <b>Note:</b> Change to a web3 provider you have in `params` below.
</div>


In [1]:
%reload_ext credmark.cmf.ipython

cmf_param = {
    'chain_id': 1,
    'block_number': None,
    # 'chain_to_provider_url': {'1': 'https://mainnet.infura.io/v3/... or https://eth-mainnet.g.alchemy.com/'}, # or, use the credmark's nodes
    'api_url': None,
    'use_local_models': None,
    'register_utility_global': True}

context, model_loader = %cmf cmf_param

## Convex Token


In [2]:
convex_token = Token(address='0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B')

with context.ledger.TokenTransfer as q:
    df_convex = (q.select(
        columns=[q.BLOCK_NUMBER,
                 q.LOG_INDEX,
                 q.TO_ADDRESS,
                 q.FROM_ADDRESS,
                 q.TOKEN_ADDRESS],
        where=q.TOKEN_ADDRESS.eq(convex_token.address),
    ).to_dataframe().sort_values(['block_number', 'log_index']))
df_convex

,block_number,log_index,to_address,from_address,token_address
3637,12451002,50,0x947b7742c403f20e5faccdac5e092c943e7d0277,0x0000000000000000000000000000000000000000,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
3848,12451232,19,0x5f465e9fcffc217c5849906216581a657cd60605,0x947b7742c403f20e5faccdac5e092c943e7d0277,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
3849,12451257,33,0xe98984ad858075813ada4261af47e68a64e28fcc,0x947b7742c403f20e5faccdac5e092c943e7d0277,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
3850,12451289,69,0x2e088a0a19dda628b4304301d1ea70b114e4accd,0x947b7742c403f20e5faccdac5e092c943e7d0277,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
3851,12451311,8,0x05767d9ef41dc40689678ffca0608878fb3de906,0x947b7742c403f20e5faccdac5e092c943e7d0277,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
...,...,...,...,...,...
1224,16648751,5,0xb576491f1e6e5e62f1d8f26062ee822b40b0e0d4,0x9507c04b10486547584c37bcbd931b2a4fee9a41,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
1225,16648763,116,0xaa0c3f5f7dfd688c6e646f66cd2a6b66acdbe434,0x0000000000000000000000000000000000000000,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
1226,16648763,120,0xaa0c3f5f7dfd688c6e646f66cd2a6b66acdbe434,0x449f2fd99174e1785cf2a1c79e665fec3dd1ddc6,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
1228,16648795,274,0xa77c88abcaa770c54a6cfbfd0c586a475537bbc1,0xe5ac0b4832d027feb1e91deeba5a79c7c2c83900,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b


## Account


In [3]:
acc_1 = Account(address=Address('0x5291fBB0ee9F51225f0928Ff6a83108c86327636'))

### Tokens that have been owned by the account


In [4]:
with context.ledger.TokenBalance as q:
    df_balance = q.select(
        where=q.ADDRESS.eq(acc_1.address),
        group_by=[q.TOKEN_ADDRESS]
    ).to_dataframe()

df_balance

,token_address
0,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
1,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
3,0xc4ad29ba4b3c580e6d59105fff484999997675ff
4,0xd533a949740bb3306d119cc777fa900ba034cd52
5,0xdac17f958d2ee523a2206206994597c13d831ec7
6,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d


### Token Transfers for this account


In [5]:
with context.ledger.TokenTransfer as q:
    group_cols = [q.BLOCK_NUMBER,
                  q.LOG_INDEX,
                  q.TO_ADDRESS,
                  q.FROM_ADDRESS,
                  q.TRANSACTION_HASH,
                  q.TOKEN_ADDRESS]

    df_tt = (q.select(
        aggregates=[
            (f'sum(case when {q.TO_ADDRESS.eq(acc_1.address)} THEN {q.VALUE} else -{q.VALUE} END)',
             'sum_value')],
        where=(q.TO_ADDRESS.eq(acc_1.address).or_(q.FROM_ADDRESS.eq(acc_1.address))),
        group_by=group_cols
    )
        .to_dataframe()
        .sort_values(['block_number', 'log_index']).reset_index())

df_tt

,index,block_number,log_index,to_address,from_address,transaction_hash,token_address,sum_value
0,0,14407839,58,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0xa9b26c2ba2e3a3176134be88bf1f95d7c7304aed,0x8f9d945275587b994caaf36656996f2012c6cf07b50f...,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,1068317764
1,1,14634606,220,0x93054188d876f558f4a66b2ef1d97d16edf0895b,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x2fbacf618cf121a2d74da84f6a1bc2697b031da48354...,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,-750000000
2,2,14634606,222,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x93054188d876f558f4a66b2ef1d97d16edf0895b,0x2fbacf618cf121a2d74da84f6a1bc2697b031da48354...,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,748139023
3,3,14649241,364,0x3993d34e7e99abf6b6f367309975d1360222d446,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0xf97d756f37c333657f425505a61d9e3c12329a6b69aa...,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,-748139023
4,4,14649241,369,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x3993d34e7e99abf6b6f367309975d1360222d446,0xf97d756f37c333657f425505a61d9e3c12329a6b69aa...,0xc4ad29ba4b3c580e6d59105fff484999997675ff,199997895019529326442
5,5,14649612,609,0x989aeb4d175e16225e39e87d0d97a3360524ad80,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x96e6c23311919875b58c5324c976838192772d8fc236...,0xc4ad29ba4b3c580e6d59105fff484999997675ff,-199997895019529326442
6,6,15672527,325,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x9e3feaf5f0483b2e196db31635734f627fdfd254,0x3dbaf50cdb30395c651500ef13592a109b8eca9f97fe...,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,100000
7,7,15672537,239,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x9e3feaf5f0483b2e196db31635734f627fdfd254,0x32f07bfee2a3382fe752e2599b8354627bbe9511ee52...,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,942140842
8,8,15676502,281,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x8ec4b391596ceed7e588cfd8bf7725d02aa42ab3a286...,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,-1000000
9,9,15676502,291,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0x8ec4b391596ceed7e588cfd8bf7725d02aa42ab3a286...,0xdac17f958d2ee523a2206206994597c13d831ec7,196895601


### The Tokens


In [6]:
wBTC = Token(address='0x2260fac5e5542a773aa44fbcfedf7c193bc2c599')
wBTC.scaled(748139023)

renBTC = Token(address='0xeb4c2781e4eba804ce9a9803c67d0893436bb27d')
renBTC

t1 = Token(address='0xdac17f958d2ee523a2206206994597c13d831ec7')
t2 = Token(address='0x2260fac5e5542a773aa44fbcfedf7c193bc2c599')
t3 = Token(address='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')

In [7]:
curve_usd_btc_eth = Token(address='0xc4ad29ba4b3c580e6d59105fff484999997675ff')
print(curve_usd_btc_eth.balance_of(acc_1.address))

ve_crv = Token(address='0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2')
print(ve_crv.balance_of(acc_1.address))
print(ve_crv.functions.locked(acc_1.address).call())

convex_voter_proxy = Contract(address='0x989AEb4d175e16225E39E87d0D97A3360524AD80')
convex_voter_proxy.abi.functions

0
0
[0, 0]


Functions: [balanceOfPool, claimCrv, claimFees, claimRewards, createLock, crv, deposit, depositor, escrow, execute, gaugeController, getName, increaseAmount, increaseTime, mintr, operator, owner, release, setDepositor, setOperator, setOwner, setStashAccess, vote, voteGaugeWeight, withdraw, withdrawAll]

In [8]:
from decimal import Decimal
pos_dict = {}
for n, r in df_tt.iterrows():
    tok_address = r['token_address']
    value = r['sum_value']
    pos_dict[tok_address] = pos_dict.get(tok_address, Decimal(0)) + Decimal(value)
    if pos_dict[tok_address] == 0:
        tok = Token(address=tok_address)
        print(tok.symbol, tok_address)
        del pos_dict[tok_address]

display(pos_dict)

token_dict = {}
for tok_address, amount in pos_dict.items():
    tok = Token(address=tok_address)
    token_dict[tok_address] = tok
    pos_dict[tok_address] = tok.scaled(amount)

pos_dict

WBTC 0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
crv3crypto 0xc4ad29ba4b3c580e6d59105fff484999997675ff
renBTC 0xeb4c2781e4eba804ce9a9803c67d0893436bb27d
crv3crypto 0xc4ad29ba4b3c580e6d59105fff484999997675ff
crv3crypto 0xc4ad29ba4b3c580e6d59105fff484999997675ff
crv3crypto 0xc4ad29ba4b3c580e6d59105fff484999997675ff
CRV 0xd533a949740bb3306d119cc777fa900ba034cd52
USDT 0xdac17f958d2ee523a2206206994597c13d831ec7
CVX 0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
crv3crypto 0xc4ad29ba4b3c580e6d59105fff484999997675ff


{'0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': Decimal('2169356'),
 '0xdac17f958d2ee523a2206206994597c13d831ec7': Decimal('479354369')}

{'0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': Decimal('2.169356'),
 '0xdac17f958d2ee523a2206206994597c13d831ec7': Decimal('479.354369')}

### Curve Pool


In [9]:
pool_info = context.models.curve_fi.pool_info(address='0xc4ad29ba4b3c580e6d59105fff484999997675ff')
pool_info = context.models.curve_fi.pool_info(address=pool_info['address'])
pool_info

In [ ]:
pool = Contract(address='0xd51a44d3fae010294c616388b506acda1bfaae46')
b1, b2, b3 = (pool.functions.balances(0).call(),
              pool.functions.balances(1).call(),
              pool.functions.balances(2).call())
pool

In [ ]:
cc = Contract(address=pool_info['gauges']['accounts'][0]['address'])
print(cc.functions.balanceOf(acc_1.address.checksum).call())
print(cc.functions.working_balances(acc_1.address.checksum).call())
print(cc.functions.claimable_tokens(acc_1.address.checksum).call())